In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok jinja2 groq diffusers transformers accelerate safetensors

In [ ]:
                                                                                                                                                                                                      import nest_asyncio
import os
from pyngrok import ngrok
from fastapi import FastAPI, Request, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from groq import Groq
from PIL import Image
import io, json, base64
import torch
from diffusers import AutoPipelineForText2Image, DEISMultistepScheduler
from fastapi.middleware.cors import CORSMiddleware
import uvicorn

In [ ]:
# Required to allow nested event loops in Colab
nest_asyncio.apply()


os.environ["GROQ_API_KEY"] = "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"


In [ ]:
# Load image generation model
pipe = AutoPipelineForText2Image.from_pretrained('lykon/dreamshaper-8', torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DEISMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")
pipe.enable_attention_slicing()
pipe.enable_vae_tiling()

# Initialize FastAPI
app = FastAPI()

# Templates directory
templates = Jinja2Templates(directory="templates")

# Groq Client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:30: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [ ]:
storyboard_system_prompt = """You are a visual assistant specialized in Storyboard Generation for image generation models.

You are using the DreamShaper 8 model (lykon/dreamshaper-8), which excels at anime-inspired, fantasy, and semi-realistic illustrations with vibrant colors and cinematic quality. Avoid photorealism, modern realism, or harsh lighting. Focus on dreamlike, storybook, painterly, or fantasy-themed visuals. Do not include text or labels in images.

When the user provides a story prompt, follow these instructions:

1. Determine if it describes a story, journey, or process.

2. Split the story into 4 logical, visual scenes that progress meaningfully.

For each scene:
- Write a clear, standalone, image-friendly prompt.
- Always include the subject, setting, mood, and style in each prompt.
- Do NOT use pronouns like "he", "she", or "they".
- Use rich visual descriptors: lighting, color palette, atmosphere, motion, fantasy elements.
- Include keywords helpful for DreamShaper-8: “fantasy setting”, “anime style”, “dramatic lighting”, “glowing particles”, “high detail”, “surreal background”, etc.
- Keep prompts under 30 words where possible.
- Avoid narrative exposition — describe what should visually appear.

Output ONLY in this format:

{
  "category": "Storyboard Generation",
  "scenes": [
    "<precise image-ready prompt>",
    "<precise image-ready prompt>",
    "<precise image-ready prompt>",
    "<precise image-ready prompt>"
  ]
}

Do not explain anything. Do not include any text outside the JSON.
"""


In [ ]:
from fastapi.responses import JSONResponse

@app.post("/generate-storyboard")
async def generate_storyboard(request: Request, prompt: str = Form(...)):
    # 1. Call LLM to split into scenes
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {"role": "system", "content": storyboard_system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=False
    )

    try:
        result = json.loads(response.choices[0].message.content)
        scene_prompts = result.get("scenes", [])
    except Exception as e:
        return JSONResponse(content={"error": f"Failed to parse LLM output: {str(e)}"}, status_code=500)

    # 2. Generate one image per scene
    images = []
    for scene in scene_prompts:
        img = pipe(scene, height=512, width=512, guidance_scale=8.5, num_inference_steps=50).images[0]
        images.append(img)

    cols =2
    padding =10
    rows = 2
    bg_color = (255, 255, 255)
 # Get image size (assume all same size)
    img_width, img_height = images[0].size

    # Create canvas
    storyboard_width = cols * img_width + (cols + 1) * padding
    storyboard_height = rows * img_height + (rows + 1) * padding
    storyboard = Image.new("RGB", (storyboard_width, storyboard_height), bg_color)

    # Paste images
    for idx, img in enumerate(images):
        row = idx // cols
        col = idx % cols
        x = padding + col * (img_width + padding)
        y = padding + row * (img_height + padding)
        storyboard.paste(img, (x, y))


    # 4. Convert to base64
    buf = io.BytesIO()
    storyboard.save(buf, format="PNG")
    img_b64 = base64.b64encode(buf.getvalue()).decode()

    return JSONResponse(content={"storyboard": img_b64})


@app.post("/generate-illustration")
async def generate_illustration(request: Request, prompt: str = Form(...)):
    image = pipe(prompt, height=512, width=512, guidance_scale=10, num_inference_steps=50).images[0]

    buf = io.BytesIO()
    image.save(buf, format="PNG")
    img_b64 = base64.b64encode(buf.getvalue()).decode()

    return JSONResponse(content={"illustration": img_b64})


In [ ]:
os.makedirs("templates", exist_ok=True)

with open("templates/index.html", "w") as f:
    f.write("""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Storyboard Generator</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background-color: #f2f4f8;
            margin: 0;
            padding: 0;
            display: flex;
            align-items: center;
            justify-content: center;
            height: 100vh;
        }
        .container {
            background: white;
            padding: 40px;
            border-radius: 12px;
            box-shadow: 0 8px 20px rgba(0, 0, 0, 0.1);
            max-width: 600px;
            width: 100%;
        }
        h2 {
            margin-bottom: 20px;
            color: #333;
        }
        textarea {
            width: 100%;
            padding: 15px;
            border: 1px solid #ccc;
            border-radius: 8px;
            resize: vertical;
            font-size: 16px;
        }
        .button-group {
            display: flex;
            justify-content: space-between;
            margin-top: 20px;
        }
        .button-group button {
            flex: 1;
            padding: 12px;
            background-color: #4a90e2;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            font-size: 16px;
            margin: 0 5px;
            transition: background-color 0.3s ease;
        }
        .button-group button:hover {
            background-color: #357ab8;
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>🎬 Generate Visual Content</h2>
        <form id="promptForm" method="post">
            <textarea name="prompt" rows="5" placeholder="E.g., A teenage boy finds a magical sword hidden in the forest..."></textarea><br>
            <div class="button-group">
                <button type="submit" formaction="/generate-storyboard">📽️ Storyboard</button>
                <button type="submit" formaction="/generate-illustration">🎨 Illustration</button>
            </div>
        </form>
    </div>
</body>
</html>
""")

with open("templates/result.html", "w") as f:
    f.write("""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Generated Storyboard</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background-color: #f9fafc;
            margin: 0;
            padding: 20px;
            text-align: center;
        }

        .container {
            max-width: 900px;
            margin: auto;
            background: white;
            padding: 30px;
            border-radius: 12px;
            box-shadow: 0 6px 20px rgba(0, 0, 0, 0.08);
        }

        h2 {
            color: #2c3e50;
            margin-bottom: 20px;
        }

        .image-scroll-container {
            overflow-x: auto;
            overflow-y: hidden;
            width: 100%;
            max-width: 800px;
            margin: 0 auto;
            border-radius: 10px;
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
        }

        .image-scroll-container img {
            height: 512px;
            width: auto;
            display: block;
        }

        .back-link {
            display: inline-block;
            margin-top: 25px;
            text-decoration: none;
            color: #4a90e2;
            font-weight: 500;
        }

        .back-link:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>📖 Your Generated Storyboard</h2>
        <div class="image-scroll-container">
            <img src="data:image/png;base64,{{ storyboard }}" alt="Storyboard Image">
        </div>
        <a class="back-link" href="/">← Generate Another</a>
    </div>
</body>
</html>
""")

In [ ]:
# Add your token below
ngrok.set_auth_token("30MAhiisD1xn5S97n4vflKuRSYc_7r2Fp4bzgtaUNW6Ui6kQx")

#Allow CORS if needed
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print(f"Open this in your browser: {public_url}")

# Run the FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)

Open this in your browser: NgrokTunnel: "https://33480331dd63.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [8599]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     45.122.120.19:0 - "OPTIONS /generate-storyboard HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     45.122.120.19:0 - "POST /generate-storyboard HTTP/1.1" 200 OK


RuntimeError: Event loop stopped before Future completed.